In [54]:
from selenium import webdriver
from time import sleep
import pandas as pd
from io import StringIO
from functools import partial
import matplotlib.pyplot as plt
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException,StaleElementReferenceException
from matplotlib import style
from selenium.webdriver.support.ui import WebDriverWait

In [26]:
#打开网页
driver = webdriver.Chrome()
url = 'https://zu.fang.com/'
driver.get(url)

In [36]:
#debug
table =driver.find_element('class name','houseList')
    # Extract table rows
rows = table.find_elements(By.TAG_NAME, 'dl')

In [38]:
table.text

'直播看房\n正南 贻成学府壹号 2室1厅 精装修\n整租|2室1厅|83㎡|朝南\n滨海新区-中新生态城-贻成学府壹号\n交通便利精装修采光好\n2800元/月\n直播看房\n南北通透 碧桂园滨海城 3室1厅 精装修\n整租|3室1厅|94㎡|朝南北\n滨海新区-中新生态城-碧桂园滨海城\n拎包入住采光好精装修\n3200元/月\n直播看房\n宸和园 2室2厅1卫 电梯房 精装\n整租|3室2厅|135㎡|朝南北\n滨海新区-中新生态城-宸和园\n生活便利采光好拎包入住\n4000元/月\n直播看房\n正南 吉宝季景铭郡 2室2厅 精装修\n整租|2室2厅|87㎡|朝南\n滨海新区-中新生态城-吉宝季景铭郡\n生活便利采光好拎包入住\n2300元/月\n直播看房\n南北通透 融创观澜壹號 2室2厅 精装修\n整租|2室2厅|80㎡|朝南北\n滨海新区-中新生态城-融创观澜壹號\n拎包入住精装修生活便利\n2800元/月\n直播看房\n南北通透鲲玺园3室2厅精装修\n整租|3室2厅|145㎡|朝南北\n滨海新区-中新生态城-鲲玺园\n精装修拎包入住生活便利\n2800元/月\n直播看房\n南北通透 首创康桥郡 2室2厅 精装修\n整租|2室2厅|94㎡|朝南北\n滨海新区-中新生态城-首创康桥郡\n拎包入住精装修生活便利\n3200元/月\n直播看房\n正南鲲贝园2室1厅精装修\n整租|2室1厅|92㎡|朝南\n滨海新区-中新生态城-鲲贝园\n拎包入住采光好精装修\n2200元/月\n直播看房\n正南 家和园 2室2厅 精装修\n整租|2室2厅|99㎡|朝南\n滨海新区-中新生态城-家和园\n生活便利拎包入住精装修\n2500元/月\n直播看房\n南北通透凤栖梧桐2室2厅精装修\n整租|2室2厅|99㎡|朝南北\n滨海新区-中新生态城-凤栖梧桐\n精装修拎包入住生活便利\n2700元/月\n直播看房\n南北通透和溪园3室2厅精装修\n整租|3室2厅|97㎡|朝南北\n滨海新区-中新生态城-和溪园\n精装修拎包入住生活便利\n2800元/月\n南北通透亿利国际生态岛3室2厅精装修\n整租|3室2厅|150㎡|朝南北\n滨海新区-中新生态城-亿利国际生态岛\n采光好精装修拎包入住\n5000元/月\n正南鲲贝园2室2厅精装修\n整租|2室2厅|92㎡|朝南\n滨海新区-

In [69]:
string_list = []

i = 0

while i < 20:
    #爬数据
    table =driver.find_element('class name','houseList')
    rows = table.find_elements(By.TAG_NAME, 'dl')

    # 收拾数据
    data = []
    for row in rows:
        title = row.find_element(By.CSS_SELECTOR, "p.title").text
        details_text = row.find_element(By.CSS_SELECTOR, "p.font15.mt12.bold").text
        details = details_text.split("|")
        if len(details) >= 4:
            rent_type = details[0]  
            floor = details[1]
            area = details[2]         
            direction = details[3]
        else:
            house_type = area = floor = direction = None 
        #locate = row.find_element(By.CSS_SELECTOR, "p#rentid_D09_1_06.gray6.mt12").text
        locate = driver.find_element(By.XPATH, "//p[@id='rentid_D09_1_06' and contains(@class, 'gray6') and contains(@class, 'mt12')]").text
        price = row.find_element(By.CSS_SELECTOR, "p.mt5.alingC").text
        data.append([title, rent_type, floor, area, direction, locate, price])

    df = pd.DataFrame(data)

    #检查运行
    print(1)
    string_list.append(df)
    #翻页
    try:
        next_page = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable(
                (By.XPATH, '//div[@id="rentid_D10_01"]/a[text()="下一页"]')
            )
        )
        next_page.click()
        print("成功点击下一页")
        i+=1   
    except Exception as e:
        break

1
成功点击下一页
1
成功点击下一页
1
成功点击下一页
1
成功点击下一页
1
成功点击下一页
1
成功点击下一页
1
成功点击下一页
1


In [71]:
driver.quit()

In [72]:
#进一步收拾数据并观察
df_full=pd.concat(string_list)
df_full.columns=['标题',
'租房类型',
'房型',
'面积',
'朝向',
'位置','价格',]

In [76]:
df_full.describe()

,标题,租房类型,房型,面积,朝向,位置,价格
count,442,442,442,442,442,442,442
unique,271,1,10,91,7,8,44
top,正南力高阳光海岸逸海苑1室1厅精装修,整租,3室2厅,89㎡,朝南北,滨海新区-中新生态城-家和园,3000元/月
freq,10,442,236,30,278,60,51


In [79]:
#输出
df_full.to_csv('df_full_rent.csv', index=False)